## Deliverable 2: Create a Customer Travel Destinations Map

In [73]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import os

# Add parent folder to Python sys.path in order to import the config module
parent_folder = os.path.dirname(os.getcwd())
sys.path.append(parent_folder)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [74]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Boston,US,42.3584,-71.0598,73.58,59,57,7.00,broken clouds
1,New York,US,40.7143,-74.0060,79.95,47,1,11.50,clear sky
2,Bethesda,US,38.9807,-77.1003,80.22,47,1,8.01,clear sky
3,Chicago,US,41.8500,-87.6500,82.99,47,4,4.00,clear sky
4,Tampa,US,27.9475,-82.4584,88.95,56,75,1.01,broken clouds


In [75]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 96


In [76]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Boston,US,42.3584,-71.0598,73.58,59,57,7.00,broken clouds
1,New York,US,40.7143,-74.0060,79.95,47,1,11.50,clear sky
2,Bethesda,US,38.9807,-77.1003,80.22,47,1,8.01,clear sky
3,Chicago,US,41.8500,-87.6500,82.99,47,4,4.00,clear sky
4,Tampa,US,27.9475,-82.4584,88.95,56,75,1.01,broken clouds


In [77]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [78]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [79]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boston,US,73.58,broken clouds,42.3584,-71.0598,
1,New York,US,79.95,clear sky,40.7143,-74.0060,
2,Bethesda,US,80.22,clear sky,38.9807,-77.1003,
3,Chicago,US,82.99,clear sky,41.8500,-87.6500,
4,Tampa,US,88.95,broken clouds,27.9475,-82.4584,


In [80]:
import json

In [81]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # add location to params
    params["location"] = f"{lat},{lng}"
    # print(params)
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_json = requests.get(base_url, params).json()
    
    # print(json.dumps(hotel_json["results"][0], indent=4))
    # print(hotel_json["results"][0]["name"])
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_json["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        continue 

In [82]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Boston,US,73.58,broken clouds,42.3584,-71.0598,Kimpton Nine Zero Hotel
1,New York,US,79.95,clear sky,40.7143,-74.0060,Duane Street Hotel
2,Bethesda,US,80.22,clear sky,38.9807,-77.1003,Hyatt Regency Bethesda
3,Chicago,US,82.99,clear sky,41.8500,-87.6500,Chinatown Hotel
4,Tampa,US,88.95,broken clouds,27.9475,-82.4584,Hilton Tampa Downtown


In [83]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [84]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.set_index("City", inplace=True)
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [85]:
# reset index
clean_hotel_df.reset_index(inplace=True)

In [86]:
clean_hotel_df.columns

Index(['City', 'Country', 'Max Temp', 'Current Description', 'Lat', 'Lng',
       'Hotel Name'],
      dtype='object')

In [87]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [88]:
def create_heat_map(locations, weights, marker_info=None):
    """
    create new heat map with gmaps for locations and weights passed in
    
    args:
        locations: DataFrame with two columns [["Lat", "Lng"]]
        weights: List of max temperatures or humidity, or another weather metric
    """ 
    # create figure
    fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
    
    heat_layer = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=300, point_radius=4)
        
    # add the heatmap layers
    fig.add_layer(heat_layer)

    # add marker layer if marker_info argument is provided
    if (marker_info != None):
        marker_layer = gmaps.marker_layer(locations, info_box_content=marker_info)
        fig.add_layer(marker_layer)

    
    # call figure to plot the data
    return fig

In [89]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = create_heat_map(locations, max_temp, hotel_info)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))